Work in progress...

In [ ]:
import pandas as pd
from jira import JIRA

jira = JIRA('https://kainos-evolve.atlassian.net')
jql = 'project=VXT and type=epic and status != Completed'

epicsRaw = jira.search_issues(jql)

epics = pd.DataFrame()
epics['version'] = ''
epics['key'] = ''
epics['type'] = ''
epics['status'] = ''
epics['summary'] = ''

#add epics to dataframe
for issue in epicsRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        epics = epics.append(
            {'version': fixVersion.name, 
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'summary': issue.fields.summary,
            }, ignore_index=True)
            
epics

In [ ]:
issues = pd.DataFrame()
issues['version'] = ''
issues['key'] = ''
issues['type'] = ''
issues['status'] = ''
issues['SP'] = 0
issues['summary'] = ''
